dbt (data build tool) is one of the hottest technologies in the data engineering and analytics space.

### Step 1: Create a dbt project
We will be populating some data in a Postgres database therefore, we first need to install the dbt Postgres adapter from PyPI:

In [ ]:
pip install dbt-postgres==1.3.1

Note that the command will also install the dbt-core package as well as other dependencies that are required for running dbt.

Now let’s go ahead and create a dbt project — to do so, we can initialise a new dbt project by running the dbt init command in the terminal:

In [ ]:
dbt init test_dbt_project

Then we will be prompted to select which database we like to use (depending on the adapters we have installed locally, we may see different options)

**Step 2: Create a Docker Compose file**

Now let’s create a docker-compose.yml file (place the file at the same level as the test_dbt_projectdirectory) in which we will be specifying two services — one would correspond to a ready-made Postgres image and the second one to a dbt image that we will define in a Dockerfile in the next step:

In [ ]:
version: "3.9"

services:
  postgres:
    container_name: postgres
    image: frantiseks/postgres-sakila
    ports:
      - '5432:5432'
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 5s
      timeout: 5s
      retries: 5
  dbt:
    container_name: dbt
    build: .
    image: dbt-dummy
    volumes:
      - ./:/usr/src/dbt
    depends_on:
      postgres:
        condition: service_healthy

As you can tell, for the Postgres container, we will be using an image called frantiseks/postgres-sakila which is publicly available and accessible on Docker Hub. This image, will populate the Sakila Database on the Postgres instance. The database models a DVD rental store and is consisted of multiple tables which are normalised and correspond to entities such as films, actors, customers and payments. In the next few following sections we’ll make use of this data in order to build some example dbt data models.

The second service, called dbt, will be the one that creates an environment where we will build our data models. Note that we mount the current directory into the docker container. This will let the container have access to any changes we may be doing to the data models without having to re-build the image. Additionally, any metadata generated by dbt commands (such as manifet.json) will appear instantly on the host machine.

**Step 3: Create a Dockerfile.**

Now let’s specify a Dockerfile that will be used to build an image on top of which the running container will then build the models specified in our example dbt project.

In [ ]:
FROM python:3.10-slim-buster

RUN apt-get update \
    && apt-get install -y --no-install-recommends

WORKDIR /usr/src/dbt/dbt_project

# Install the dbt Postgres adapter. This step will also install dbt-core
RUN pip install --upgrade pip
RUN pip install dbt-postgres==1.3.1

# Install dbt dependencies (as specified in packages.yml file)
# Build seeds, models and snapshots (and run tests wherever applicable)
CMD dbt deps && dbt build --profiles-dir profiles && sleep infinity

Note that in the last CMD command, we intentionally added an extra && sleep infinity command such that the container won’t exit after running the steps specified in the Dockerfile so that we can then access the container and run additional dbt commands (if needed).

**Step 4: Create a dbt profile for the Postgres database.**

Now that we have created the required infrastructure for our host machines in order to create a Postgres database, populate some dummy data as well as creating an image for our dbt environment, let’s focus on the dbt side.

We will first have to create a dbt profile that will be used when interacting with the target Postgres database. Within the test_dbt_project directory, create another directory called profiles and then a file called profiles.yml with the following content:

In [ ]:
test_profile:
  target: dev
  outputs:
    dev:
      type: postgres
      host: postgres
      user: postgres
      password: postgres
      port: 5432
      dbname: postgres
      schema: public
      threads: 1

**Step 5: Define some data models.**

The next step is to create some data models based on the Sakila data populated by the Postgres container. If you are planning to use this project for testing purposes, I would advise to create at least one seed, one model and a snapshot (with tests if possible) so that you have a full coverage of all dbt entities (macros excluding).

**Step 6: Run the Docker containers.**

We now have everything we need in order to spin up the two docker containers we specified in the docker-compose.yml file earlier, and build the data models defined in our example dbt project.

First, let’s build the images

In [ ]:
docker-compose build

And now let’s spin up the running containers:

In [ ]:
docker-compose up

This command should have initialised a Postgres database using the Sakila Database, and created the dbt models specified. For now, let’s make sure you have two running containers:

In [ ]:
docker ps

should give an output that includes one container with name dbt and another one with name *postgres*.

**Step 7: Query the models on Postgres database.**

In order to access the Postgres container, you’ll first need to infer the container id

In [ ]:
docker ps

And then run

In [ ]:
docker exec -it <container-id> /bin/bash

We will then need to use psql, a command-line interface that gives us access the postgres instance:

In [ ]:
psql -U postgres

If you have used the data models I’ve shared in the previous sections, you can now query each of the models created on Postgres using the queries below.

In [ ]:
-- Query seed tables
SELECT * FROM customer_base;

-- Query staging views
SELECT * FROM stg_payment;

-- Query intermediate views
SELECT * FROM int_customers_per_store;
SELECT * FROM int_revenue_by_date;

-- Query mart tables
SELECT * FROM cumulative_revenue;

-- Query snapshot tables
SELECT * FROM int_stock_balances_daily_grouped_by_day_snapshot;

**Step 8: Creating additional or modifying existing models**


As mentioned already, the Dockerfile and docker-compose.yml files were written in such a way such that the dbt container would still be up and running. Therefore, whenever you modify or create data models, you can still use that container to re-build seeds, models, snapshots and/or tests.

To do so, first infer the container id of the dbt container:

In [ ]:
docker ps

Then enter the running container by running

In [ ]:
docker exec -it <container-id> /bin/bash

And finally run any dbt command you wish, depending on the modifications you’ve made to the example dbt project. Here’s a quick reference of the most commonly used commands for these purposes:

In [ ]:
# Install dbt deps
dbt deps

# Build seeds
dbt seeds --profiles-dir profiles

# Build data models
dbt run --profiles-dir profiles

# Build snapshots
dbt snapshot --profiles-dir profiles

# Run tests
dbt test --profiles-dir profiles